In [2]:
!pip install aif360

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.7/259.7 kB 1.8 MB/s eta 0:00:00


In [3]:
import pandas as pd
from aif360.detectors.mdss_detector import bias_scan

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# 定義 Excel 檔案的路徑
file_paths = [
    # '105年A1-A4所有當事人.xlsx',
    # '106年A1-A4所有當事人.xlsx',
    # '107年A1-A4所有當事人(新增戶籍地).xlsx',
    # '108年A1-A4所有當事人(新增戶籍地).xlsx',
    # '109年A1-A4所有當事人(新增戶籍地).xlsx',
    '/content/drive/MyDrive/dataset/105年A1-A4所有當事人.xlsx',
    '/content/drive/MyDrive/dataset/106年A1-A4所有當事人.xlsx',
    '/content/drive/MyDrive/dataset/107年A1-A4所有當事人(新增戶籍地).xlsx',
    '/content/drive/MyDrive/dataset/108年A1-A4所有當事人(新增戶籍地).xlsx',
    '/content/drive/MyDrive/dataset/109年A1-A4所有當事人(新增戶籍地).xlsx'

]

selected_columns = ['性別', '年齡', '車種', '15事故類型及型態', '22受傷程度','Delivery_Type']

# 初始化一個空的 DataFrame 用於存儲合併後的數據
df_combined = pd.DataFrame()

# 讀取每個 Excel 文件並合併到一個 DataFrame 中
for file_path in file_paths:
    df = pd.read_excel(file_path, engine='openpyxl')
    df_combined = pd.concat([df_combined, df], ignore_index=True)

# 檢查合併後的數據
print("Combined DataFrame:")
print(df_combined.head())


Combined DataFrame:
  當事人序        1 Unnamed: 2 Unnamed: 3  編號  年度 汽車駕籍地 機車駕籍地  戶籍地 發生時間  ...   X  \
0  NaN      NaN        NaN        NaN NaN NaN   NaN   NaN  NaN  NaN  ... NaN   
1  列標籤  計數 - 案號  加總 - 死亡人數  加總 - 受傷人數 NaN NaN   NaN   NaN  NaN  NaN  ... NaN   
2    1       88         89         36 NaN NaN   NaN   NaN  NaN  NaN  ... NaN   
3    2    22554          0      29313 NaN NaN   NaN   NaN  NaN  NaN  ... NaN   
4    3    16304          0          0 NaN NaN   NaN   NaN  NaN  NaN  ... NaN   

    Y   路口   路段  路段2    季  上半年 PoliceStation 肇事原因  Delivery_Type  
0 NaN  NaN  NaN  NaN  NaN  NaN           NaN  NaN            NaN  
1 NaN  NaN  NaN  NaN  NaN  NaN           NaN  NaN            NaN  
2 NaN  NaN  NaN  NaN  NaN  NaN           NaN  NaN            NaN  
3 NaN  NaN  NaN  NaN  NaN  NaN           NaN  NaN            NaN  
4 NaN  NaN  NaN  NaN  NaN  NaN           NaN  NaN            NaN  

[5 rows x 108 columns]


In [11]:
# 讀取資料
#file_path = '108年A1-A4所有當事人(新增戶籍地).xlsx'
#df = pd.read_excel(file_path, header=0, engine='openpyxl')
#print(df.head())

In [14]:
# 選擇指定欄位
df_cleaned = df[['性別', '年齡', '車種', '15事故類型及型態', '22受傷程度','Delivery_Type']].copy()

# 進行欄位轉換和清理
# '性別' 欄位：男性設為1，女性設為2，移除空白欄位
df_cleaned['性別'] = df_cleaned['性別'].replace({'1': 1, '2': 2}).dropna()
print("After replacing '性別':")
print(df_cleaned['性別'].head())

df_cleaned = df_cleaned.dropna(subset=['性別'])
print("After dropping NA '性別':")
print(df_cleaned.head())

After replacing '性別':
0    3.0
1    1.0
2    1.0
3    1.0
4    3.0
Name: 性別, dtype: float64
After dropping NA '性別':
    性別    年齡   車種  15事故類型及型態  22受傷程度  Delivery_Type
0  3.0   0.0  G06        NaN     NaN            0.0
1  1.0  55.0  H03        NaN     NaN            0.0
2  1.0  42.0  B12        NaN     NaN            0.0
3  1.0  40.0  B01        NaN     NaN            0.0
4  3.0   0.0  NaN        NaN     NaN            0.0


In [15]:
# '年齡' 欄位：18歲以下設為1，19至64歲設為2，65歲以上設為3，移除空白欄位
df_cleaned['年齡'] = pd.cut(df_cleaned['年齡'].astype(float), bins=[0, 18, 64, float('inf')], labels=[1, 2, 3], right=False)
print("After binning '年齡':")
print(df_cleaned['年齡'].head())

df_cleaned = df_cleaned.dropna(subset=['年齡'])
print("After dropping NA '年齡':")
print(df_cleaned.head())

After binning '年齡':
0    1
1    2
2    2
3    2
4    1
Name: 年齡, dtype: category
Categories (3, int64): [1 < 2 < 3]
After dropping NA '年齡':
    性別 年齡   車種  15事故類型及型態  22受傷程度  Delivery_Type
0  3.0  1  G06        NaN     NaN            0.0
1  1.0  2  H03        NaN     NaN            0.0
2  1.0  2  B12        NaN     NaN            0.0
3  1.0  2  B01        NaN     NaN            0.0
4  3.0  1  NaN        NaN     NaN            0.0


In [17]:
# '車種' 欄位：B03設為1，C03設為2，其他全部設為0
df_cleaned['車種'] = df_cleaned['車種'].apply(lambda x: 1 if x == 'B03' else (2 if x == 'C03' else 0))
print("After replacing '車種':")
print(df_cleaned['車種'].head())

df_cleaned = df_cleaned.dropna(subset=['車種'])
print("After dropping NA '車種':")
print(df_cleaned.head())

After replacing '車種':
0    0
1    0
2    0
3    0
4    0
Name: 車種, dtype: int64
After dropping NA '車種':
    性別 年齡  車種  15事故類型及型態  22受傷程度  Delivery_Type
0  3.0  1   0        NaN     NaN              0
1  1.0  2   0        NaN     NaN              0
2  1.0  2   0        NaN     NaN              0
3  1.0  2   0        NaN     NaN              0
4  3.0  1   0        NaN     NaN              0


In [18]:
# 'Delivery_Type' 欄位：01.Foodpanda設成1, 02.UberEats設成2 其他設為0
df_cleaned['Delivery_Type'] = df_cleaned['Delivery_Type'].apply(lambda x: 1 if x == '01' else (2 if x == '02' else 0))
print("After replacing 'Delivery_Type':")
print(df_cleaned['Delivery_Type'].head())

df_cleaned = df_cleaned.dropna(subset=['Delivery_Type'])
print("After dropping NA 'Delivery_Type':")
print(df_cleaned.head())

After replacing 'Delivery_Type':
0    0
1    0
2    0
3    0
4    0
Name: Delivery_Type, dtype: int64
After dropping NA 'Delivery_Type':
    性別 年齡  車種  15事故類型及型態  22受傷程度  Delivery_Type
0  3.0  1   0        NaN     NaN              0
1  1.0  2   0        NaN     NaN              0
2  1.0  2   0        NaN     NaN              0
3  1.0  2   0        NaN     NaN              0
4  3.0  1   0        NaN     NaN              0


In [19]:
# '15事故類型及型態' 欄位：空白欄位設為0，非空白欄位設為1
df_cleaned['15事故類型及型態'] = df_cleaned['15事故類型及型態'].notna().astype(int)
print("After processing '15事故類型及型態':")
print(df_cleaned['15事故類型及型態'].head())

df_cleaned = df_cleaned.dropna(subset=['15事故類型及型態'])
print("After dropping NA '15事故類型及型態':")
print(df_cleaned.head())

After processing '15事故類型及型態':
0    0
1    0
2    0
3    0
4    0
Name: 15事故類型及型態, dtype: int64
After dropping NA '15事故類型及型態':
    性別 年齡  車種  15事故類型及型態  22受傷程度  Delivery_Type
0  3.0  1   0          0     NaN              0
1  1.0  2   0          0     NaN              0
2  1.0  2   0          0     NaN              0
3  1.0  2   0          0     NaN              0
4  3.0  1   0          0     NaN              0


In [20]:
# '22受傷程度' 欄位：3設為1(無受傷)，其他全部設為0
df_cleaned['22受傷程度'] = (df_cleaned['22受傷程度'] == 3).astype(int)
print("After processing '22受傷程度':")
print(df_cleaned['22受傷程度'].head())

df_cleaned = df_cleaned.dropna(subset=['22受傷程度'])
print("After dropping NA '22受傷程度':")
print(df_cleaned.head())

After processing '22受傷程度':
0    0
1    0
2    0
3    0
4    0
Name: 22受傷程度, dtype: int64
After dropping NA '22受傷程度':
    性別 年齡  車種  15事故類型及型態  22受傷程度  Delivery_Type
0  3.0  1   0          0       0              0
1  1.0  2   0          0       0              0
2  1.0  2   0          0       0              0
3  1.0  2   0          0       0              0
4  3.0  1   0          0       0              0


In [21]:
# 檢查清理後的數據
print("Cleaned DataFrame:")
print(df_cleaned.head())


Cleaned DataFrame:
    性別 年齡  車種  15事故類型及型態  22受傷程度  Delivery_Type
0  3.0  1   0          0       0              0
1  1.0  2   0          0       0              0
2  1.0  2   0          0       0              0
3  1.0  2   0          0       0              0
4  3.0  1   0          0       0              0


In [25]:
# 定義特徵和目標變數
features = df_cleaned[['性別', '年齡', '車種', '15事故類型及型態','Delivery_Type']]  #'性別', '年齡', '車種', '15事故類型及型態' ,'Delivery_Type' ,'外送員*受傷程度'
y = df_cleaned['22受傷程度']

# 檢查特徵和目標變數
print("Features:")
print(features.head())
print("Target:")
print(y.head())

Features:
    性別 年齡  車種  15事故類型及型態  Delivery_Type
0  3.0  1   0          0              0
1  1.0  2   0          0              0
2  1.0  2   0          0              0
3  1.0  2   0          0              0
4  3.0  1   0          0              0
Target:
0    0
1    0
2    0
3    0
4    0
Name: 22受傷程度, dtype: int64


In [32]:
# 定義特徵和目標變數
features = df_cleaned[['性別', '年齡', '車種', '15事故類型及型態','Delivery_Type']]
y = df_cleaned['22受傷程度']
# 設置 favorable_value 為 'high'，表示 "無受傷" 是有利結果
favorable_value = 'high'

# 執行 bias_scan 函數
privileged_subset = bias_scan(data=features, observations=y, favorable_value=favorable_value,
                              scoring='BerkJones', overpredicted=True, penalty=50,
                              mode='continuous', alpha=0.1)  # 調整 alpha 值

unprivileged_subset = bias_scan(data=features, observations=y, favorable_value=favorable_value,
                                scoring='BerkJones', overpredicted=False, penalty=50,
                                mode='continuous', alpha=0.1)  # 調整 alpha 值

In [31]:
# 檢測特權群體
print("Privileged Subset:")
print(privileged_subset)
if privileged_subset[0]:
    to_choose = features[privileged_subset[0].keys()].isin(privileged_subset[0]).all(axis=1)
    temp_df = df_cleaned.loc[to_choose].copy()
    print("Our detected privileged group has a size of {}, we observe {} as the mean injury degree, but our model predicts {}"
          .format(len(temp_df), temp_df['22受傷程度'].mean(), y.mean()))
else:
    print("No privileged subset detected.")

# 檢查非特權群體
print("Unprivileged Subset:")
print(unprivileged_subset)
if unprivileged_subset[0]:
    to_choose = features[unprivileged_subset[0].keys()].isin(unprivileged_subset[0]).all(axis=1)
    temp_df = df_cleaned.loc[to_choose].copy()
    print("Our detected unprivileged group has a size of {}, we observe {} as the mean injury degree, but our model predicts {}"
          .format(len(temp_df), temp_df['22受傷程度'].mean(), y.mean()))
else:
    print("No unprivileged subset detected.")

Privileged Subset:
({}, 22531.8731)
No privileged subset detected.
Unprivileged Subset:
({'車種': [0, 1], '性別': [1.0], '15事故類型及型態': [1], '年齡': [2, 3]}, 23665.0364)
Our detected unprivileged group has a size of 18288, we observe 0.7852143482064742 as the mean injury degree, but our model predicts 0.10233569474644034


/*
找到了一個在預測受傷程度上被低估的群體，這個群體的特徵如下：
年齡：19 至 64 歲（2）和 65 歲以上（3）
性別：男性（1.0）
事故類型：事故類型為 1(有發生事故)
這個群體的大小是 38717 人。

具體偏見：

實際情況：這個群體中有 46.18% 的人實際上受傷了。
模型預測：模型卻只預測有 10.23% 的人會受傷。
*/